## Servicio Social – Comparativa de Modelos de Visión Computacional para Detección de Contaminantes en Cosechas de Chile 🌶️

In [ ]:
import torch

print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # esto por si hay disponible una GPU si no usa la CPU
print("CUDA:", torch.cuda.is_available())

2.9.0+cu126
CUDA: True


In [1]:
# montando drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
# carpetas de las imagenes
BASE_PATH = "/content/drive/MyDrive/ServicioSocial"
DATA_YAML = os.path.join(BASE_PATH, "data.yaml")

NameError: name 'os' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Integridad de los datos

In [5]:
import os
data_types = ["train", "valid", "test"]

for split in data_types:
    images_path = os.path.join(BASE_PATH, split, "images")
    labels_path = os.path.join(BASE_PATH, split, "labels")

    num_images = len([
        f for f in os.listdir(images_path)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ])

    num_labels = len(os.listdir(labels_path))

    print(f"{split.upper()}:")
    print(f"  Imágenes: {num_images}")
    print(f"  Labels:   {num_labels}")


TRAIN:
  Imágenes: 3704
  Labels:   3704
VALID:
  Imágenes: 1236
  Labels:   1236
TEST:
  Imágenes: 1232
  Labels:   1232


### Implementacion del modelo: YoloV8

In [ ]:

from ultralytics import YOLO
import torch
import os

# instancia
model = YOLO("yolov8n.pt")  # pesos preentrenados

# hyperparametros del entrenamientos
model.train(
    data=DATA_YAML,
    epochs=50,
    imgsz=640,
    batch=16,
    device=0  # usa GPU automáticamente
)

# VALIDACIÓN
metrics = model.val()
print(metrics)

# CARGAR MEJOR MODELO

best_model_path = "runs/detect/train/weights/best.pt"
model = YOLO(best_model_path)

# INFERENCIA
results = model.predict(
    source=f"{BASE_PATH}/dataset/test/images",
    conf=0.25,
    save=True
)

print("Detección finalizada")


### Evaluacion del modelo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_box_loss(results_csv_path):
    df = pd.read_csv(results_csv_path)

    plt.figure()
    plt.plot(df["epoch"], df["train/box_loss"], label="Train Box Loss")
    plt.plot(df["epoch"], df["val/box_loss"], label="Val Box Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Box Loss")
    plt.title("Box Loss durante el entrenamiento")
    plt.legend()
    plt.grid(True)
    plt.show()
